In [30]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.11.0'

In [31]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
)

test_datagen = ImageDataGenerator(rescale=1./255)

batchSize = 16
size = 224

training_set = train_datagen.flow_from_directory(
    "data/train",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

validation_set = test_datagen.flow_from_directory(
    "data/validation",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical"
)

test_set = test_datagen.flow_from_directory(
    "data/test",
    target_size=(size, size),
    batch_size=batchSize,
    class_mode="categorical",
    shuffle=False
)

Found 168 images belonging to 3 classes.
Found 36 images belonging to 3 classes.
Found 36 images belonging to 3 classes.


In [33]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [34]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(size, size, 3))

9406464/9406464 [==============================] - 1s 0us/step


In [36]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

In [38]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
epochs = 20

history = model.fit(
    training_set,
    epochs=epochs,
    validation_data=validation_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(validation_set)
)

test_loss, test_accuracy = model.evaluate(test_set)
print("Test Loss: {:.4f}, Test Accuracy: {:.4f}".format(test_loss, test_accuracy))

Epoch 1/20
11/11 [==============================] - 4s 396ms/step - loss: 0.3706 - accuracy: 0.8690 - val_loss: 0.4094 - val_accuracy: 0.8056
Epoch 2/20
11/11 [==============================] - 4s 386ms/step - loss: 0.3391 - accuracy: 0.9107 - val_loss: 0.3731 - val_accuracy: 0.9167
Epoch 3/20
11/11 [==============================] - 4s 397ms/step - loss: 0.2859 - accuracy: 0.8929 - val_loss: 0.3520 - val_accuracy: 0.8889
Epoch 4/20
11/11 [==============================] - 4s 383ms/step - loss: 0.2758 - accuracy: 0.9107 - val_loss: 0.3227 - val_accuracy: 0.9167
Epoch 5/20
11/11 [==============================] - 4s 390ms/step - loss: 0.2542 - accuracy: 0.9226 - val_loss: 0.3095 - val_accuracy: 1.0000
Epoch 6/20
11/11 [==============================] - 4s 392ms/step - loss: 0.2628 - accuracy: 0.9048 - val_loss: 0.2929 - val_accuracy: 0.8889
Epoch 7/20
11/11 [==============================] - 4s 390ms/step - loss: 0.2371 - accuracy: 0.9286 - val_loss: 0.2770 - val_accuracy: 0.9722
Epoch 

In [41]:
import tensorflow as tf

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Perform post-training quantization
quantized_tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\reday\AppData\Local\Temp\tmpj8lwdnx6\assets


INFO:tensorflow:Assets written to: C:\Users\reday\AppData\Local\Temp\tmpj8lwdnx6\assets


In [42]:
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)